## 3-1 戦時における民間人の被害を測定する

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
sns.set(style="white", color_codes=True)
import matplotlib.pyplot as plt
import matplotlib as mpl
font = {"family":"Ricty"}
mpl.rc('font', **font)

afghan = pd.read_csv("officialSource/qss-master/MEASUREMENT/afghan.csv")
afghan.shape

# 年齢について要約
afghan.age.describe()
sns.distplot(afghan.age)
plt.title("年齢分布")
plt.show()

# 教育を受けた年数
afghan["educ.years"].describe()
sns.distplot(afghan["educ.years"])
plt.title("教育を受けた年数の分布")
plt.show()

# 就業状況
afghan.employed.describe()
afghan.employed.value_counts()

# 月収状況
afghan.income.describe()
afghan.income.value_counts(dropna=False)
afghan.groupby("income").count()

# 攻撃を受けた状況を把握(ISAF, タリバン)
pd.crosstab(afghan["violent.exp.ISAF"], afghan["violent.exp.taliban"])
pd.crosstab(afghan["violent.exp.ISAF"], afghan["violent.exp.taliban"], normalize=True, margins=True)


## 3-2 欠損データを扱う

In [ ]:
# 月収の欠損レコード数
afghan.income.isnull().sum()

# 欠損割合
afghan.income.isnull().mean()

# NAをどう扱うか？は意識できるようになった方がよい
pd.crosstab(afghan["violent.exp.ISAF"], afghan["violent.exp.taliban"], normalize=True, margins=True)
pd.crosstab(afghan["violent.exp.ISAF"].fillna("NA").astype("str"), afghan["violent.exp.taliban"].fillna("NA").astype("str"), normalize=True, margins=True)


- リストワイズ除去
	- 任意のレコードの中に、1カラムでもデータ欠損があった場合、当該レコード(1行)を除去してしまう手法。

In [ ]:
# リストワイズ除去
afghan[~afghan.isnull().any(axis=1)].shape

# income 変数が欠損している場合にのみ除去
afghan[~afghan.income.isnull()].shape

### 3.3.1 棒グラフ

In [ ]:
# ISAFの被害の割合の棒グラフ

# 割合に変換
temp = afghan["violent.exp.ISAF"].value_counts(dropna=False) / afghan["violent.exp.ISAF"].value_counts(dropna=False).sum()

sns.barplot(x=["被害なし", "被害あり", "NA"], y=temp, color="gray")
plt.title("ISFAによる民間人被害")
plt.ylabel("回答者の割合")
plt.ylim(0, 0.7)
plt.show()

In [ ]:
# タリバンの被害の割合の棒グラフ

# 割合に変換
temp = afghan["violent.exp.taliban"].value_counts(dropna=False) / afghan["violent.exp.taliban"].value_counts(dropna=False).sum()
ax = sns.barplot(x=["被害なし", "被害あり", "NA"], y=temp, color="gray")
ax.title.set_text("タリバンによる民間人被害")
plt.ylim(0, 0.7)
plt.show()

### 3.3.2 ヒストグラム

In [ ]:
# 年齢のヒストグラムを描く
# Rのヒストグラムと同じビニングにするために、ちょっと小細工してる
ax = sns.distplot(afghan.age, norm_hist=True, bins=[x if x==15 else x+1 for x in range(15, 90, 5)])
ax.title.set_text("回答者の年齢分布")
ax.xaxis.label.set_text("年齢")
ax.set_ylim(0, 0.04)
ax.figure.show()

In [ ]:
# 教育年数ヒストグラム
ax = sns.distplot(afghan["educ.years"], bins=range(afghan["educ.years"].min(), afghan["educ.years"].max() + 1, 1), norm_hist=True)
ax.title.set_text("回答者の教育程度分布")
ax.yaxis.label.set_text("密度")
ax.xaxis.label.set_text("教育を受けた年数")
# 垂直線を書く時は vlines。水平線の時は hlines
ax.vlines(afghan["educ.years"].median(), 0, 1)
# テキストを置きたい時の書き方
ax.text(afghan["educ.years"].median(), 0.5, "中央値", backgroundcolor="white",ha='left', va='top', weight='bold', color='black')
ax.figure.show()

afghan["educ.years"].median()

### 3.3.3 箱ひげ図

In [ ]:
# 一つだけ boxplot
ax = sns.boxplot(afghan.age, orient="v", color="white")
ax.title.set_text("年齢の分布")
ax.yaxis.label.set_text("年齢")
ax.figure.show()

In [ ]:
# 複数のboxplot

ax = sns.boxplot(data=afghan, x="province", y="educ.years", color="white")
ax.title.set_text("州別の教育程度")
ax.yaxis.label.set_text("教育を受けた年数")
ax.figure.show()

In [ ]:
# 州別の攻撃受けた度合いの平均
# pandasの平均は、デフォルトでdropnaしてる.(というか、nullとゼロを区別するのがデフォ)
afghan.groupby(by="province").mean()[["violent.exp.taliban", "violent.exp.ISAF"]]

In [ ]:
# プロットを画像として保存する
ax.figure.savefig("ch3/educ.png")

### 3.4.1 ランダム化の役割

In [ ]:
# アフガニスタンのデータがどの程度母集団を代表しているのかを検討してみる
afghan_village = pd.read_csv("officialSource/qss-master/MEASUREMENT/afghan-village.csv")
import matplotlib.ticker as ticker

# 01フラグに名前をつける
afghan_village.loc[afghan_village["village.surveyed"] == 0, "village.surveyed.jp"] = "非抽出" 
afghan_village.loc[afghan_village["village.surveyed"] == 1, "village.surveyed.jp"] = "抽出" 

# 村の標高の面で、抽出された村が母集団を代表しているかどうか(おなじ分布になっているか？)を見てみる
ax = sns.boxplot(y="altitude", x="village.surveyed.jp", data=afghan_village, color="white")
ax.yaxis.label.set_text("標高(メートル)")
ax.xaxis.label.set_text("")
# ｙ軸のtickを3桁カンマ区切りにする
ax.yaxis.set_major_formatter(ticker.StrMethodFormatter('{x:,.0f}'))
ax.figure.show()

# 人口の面で、抽出された村が母集団を代表しているかどうか(おなじ分布になっているか？)を見てみる
ax = sns.boxplot(y="population", x="village.surveyed.jp", data=afghan_village, color="white")
ax.figure.show()

import math
# 分布が潰れてよくわからないので、人口を対数化する
ax = sns.boxplot(x=afghan_village["village.surveyed.jp"], y=afghan_village["population"].apply(math.log), color="white")
ax.yaxis.label.set_text("対数人口")
ax.xaxis.label.set_text("")
ax.figure.show()

# 抽出、非抽出の分布がほとんど一緒なので、ちゃんと代表できてると言えそう。

## 3.6 2変量関係の要約

In [ ]:
congress = pd.read_csv("officialSource/qss-master/MEASUREMENT/congress.csv")
congress.loc[congress.party=="Democrat", "party_jp"] = "民主党"
congress.loc[congress.party=="Republican", "party_jp"] = "共和党"

fig = plt.figure(figsize=(16,6))
fig.subplots(nrows=1, ncols=2)

# 80議会の議員たちの理想点を空間投票モデルへプロット
ax = sns.scatterplot(ax=fig.get_axes()[0], x="dwnom1", y="dwnom2", hue="party_jp", style="party_jp", data=congress[congress["congress"]==80])
ax.set_ylim(-1.5, 1.5)
ax.set_xlim(-1.5, 1.5)
ax.xaxis.label.set_text("経済的リベラル/保守")
ax.yaxis.label.set_text("人種的リベラル/保守")
ax.title.set_text("第80会議")
ax.figure.show()
# 112議会の議員たちの理想点を空間投票モデルへプロット
ax2 = sns.scatterplot(ax=fig.get_axes()[1], x="dwnom1", y="dwnom2", hue="party_jp", style="party_jp", data=congress[congress["congress"]==112])
ax2.set_ylim(-1.5, 1.5)
ax2.set_xlim(-1.5, 1.5)
ax2.xaxis.label.set_text("経済的リベラル/保守")
ax2.yaxis.label.set_text("人種的リベラル/保守")
ax2.title.set_text("第112会議")
ax2.figure.show()


## 各議会ごとの経済的リベラル・保守の時系列プロット

In [ ]:
dem_medians = congress[congress["party"]!="Other"].groupby(by=["congress", "party"], as_index=False).median()
ax = sns.lineplot(x="congress", y="dwnom1", hue="party", data=dem_medians)
ax.set_xlim(78, 117)
ax.set_ylim(-1.0, 1.0)
ax.yaxis.label.set_text("DW-NOMINATEスコア(第1次元)")
ax.xaxis.label.set_text("議会会期")
ax.figure.show()

## 政治的分極化と所得の不平等の関係をプロットで見る

In [ ]:
gini = pd.read_csv("officialSource/qss-master/MEASUREMENT/USGini.csv")
temp = dem_medians.pivot(index="congress", columns="party", values="dwnom1")
temp = temp.reset_index()
# 年カラムを作成
temp["year"] = [i/10 for i in range(19475, 20135, 20)]
temp["diff"] = temp.Republican - temp.Democrat
fig = plt.figure(figsize=(16, 6))
fig.subplots(nrows=1, ncols=2)
ax = sns.scatterplot(ax=fig.get_axes()[0], x="year", y="diff", data=temp, color="gray")
ax.xaxis.label.set_text("年")
ax.yaxis.label.set_text("共和党中央値ー民主党中央値")
ax.set_title("政治的分極")
ax2 = sns.scatterplot(ax=fig.get_axes()[1], x="year", y="gini", data=gini, color="gray")
ax2.xaxis.label.set_text("年")
ax2.yaxis.label.set_text("ジニ係数")
ax2.set_title("所得の不平等")
fig.show()

In [ ]:
# 相関係数を出す 手法その1
# pandas の相関係数は、メソッドが選べて、各列総当たり戦をしてくれる。
# Null除去もリストワイズ的ではなく、ちゃんと使う列でNullがあった時のみ、除去としてくれている。
temp.loc[:, "gini"] = gini.loc[[i for i in range(1, gini.shape[0], 2)], "gini"].reset_index()
temp[["diff", "gini"]].corr()

# 相関係数を出す 手法その2 scipy で出すと帰無仮説(無相関)のp値も出してくれる
import scipy.stats
scipy.stats.pearsonr(temp["diff"], gini.loc[[i for i in range(1, gini.shape[0], 2)], "gini"])
# 相関係数が0.9418128... 、p値がすっごい小さいので、相関してると逝ってよし



In [ ]:
import probscale
# from scipy import stats

# PythonでQQプロットプロットを正確に書く方法がわからんかったので、この形で書いておく・・・
position, values = probscale.plot_pos(congress[(congress['congress']==112) & (congress['party']=='Democrat')]["dwnom2"])
position2, values2 = probscale.plot_pos(congress[(congress['congress']==112) & (congress['party']=='Republican')]["dwnom2"])

temp1 = pd.DataFrame({"x": position , "y": values, "hue":["Democrat" for i in range(1,len(values)+1)]})
temp2 = pd.DataFrame({"x": position2 , "y": values2, "hue":["Republican" for i in range(1,len(values2)+1)]})
temp = temp1.append(temp2).reset_index(drop=True)

fig = plt.figure(figsize=(8, 6))
ax = fig.subplots()
sns.scatterplot(ax = ax, x="x", y='y', hue='hue', data=temp)
fig.show()


## 3.7 クラスター化

In [ ]:
from sklearn.cluster import KMeans

# 2つのクラスターに分けてみる
# クラスターわけに利用する指標dwnom1とdwnom2は、中心点0、スケーリングも一致している指標なのでそのまま
# 利用しているが、そもそも単位が違ったり、スケーリングが違う指標を使ってクラスタわけする場合は
# それぞれの指標を z得点化(標準化)する必要がある事を忘れてはいけない！
k80two_out = KMeans(n_clusters=2).fit(congress[congress["congress"]==80][["dwnom1", "dwnom2"]])
k80two_out.cluster_centers_
k80two_out.n_iter_
k80two_out.labels_
k112two_out = KMeans(n_clusters=2).fit(congress[congress["congress"]==112][["dwnom1", "dwnom2"]])
k112two_out.cluster_centers_
k112two_out.labels_

# 各クラスターに含まれる政党ごとの観察数
pd.crosstab(congress[congress["congress"]==80]["party"], k80two_out.labels_)
pd.crosstab(congress[congress["congress"]==112]["party"], k112two_out.labels_)
# 第80議会は、各政党が混ざったグループがつくられたが、112議会では、グループ＝政党となり、前章で見た分極化が進んでいる事実と一致している


# 4つのグループに分けてみる
k80four_out = KMeans(n_clusters=4).fit(congress[congress["congress"]==80][["dwnom1", "dwnom2"]])
k112four_out = KMeans(n_clusters=4).fit(congress[congress["congress"]==112][["dwnom1", "dwnom2"]])
# 結果をビジュアライズする
congress.loc[congress["congress"]==80, "cluster"] = k80four_out.labels_
congress.loc[congress["congress"]==112, "cluster"] = k112four_out.labels_
# hueに設定したいので、文字列に変換. seaborn は hueに指定するのは文字型の数字でもエラーになるので注意
congress["cluster"] = "group_" + congress.cluster.apply(str)
fig = plt.figure(figsize=(16, 6))
ax1, ax2 = fig.subplots(1, 2)

ax1 = sns.scatterplot(ax=ax1, data=congress[congress["congress"]==80], x="dwnom1", y="dwnom2", hue="cluster", style="party_jp")
ax1.xaxis.label.set_text("経済的リベラル/保守")
ax1.yaxis.label.set_text("人種的リベラル/保守")
ax1.set_xlim(-1.6, 1.6)
ax1.set_ylim(-1.6, 1.6)
ax1.title.set_text("第80会議")
ax1 = sns.scatterplot(ax=ax1, data=pd.DataFrame(k80four_out.cluster_centers_, columns=["x", "y"]), x="x", y="y", marker="*", s=200, color="gray")

ax2 = sns.scatterplot(ax=ax2, data=congress[congress["congress"]==112], x="dwnom1", y="dwnom2", hue="cluster", style="party_jp")
ax2.xaxis.label.set_text("経済的リベラル/保守")
ax2.yaxis.label.set_text("人種的リベラル/保守")
ax2.set_xlim(-1.6, 1.6)
ax2.set_ylim(-1.6, 1.6)
ax2.title.set_text("第112会議")
ax2 = sns.scatterplot(ax=ax2, data=pd.DataFrame(k112four_out.cluster_centers_, columns=["x", "y"]), x="x", y="y", marker="*", s=200, color="gray")
fig.show()

## 可視化されたグラフから読み取れる事
- 各政党とも、2つのグループに分けられている
- 中心点の傾きが、ほぼほぼ垂直 ＝ 各政党内でのグルーピングの違いは「人種的リベラル・保守の違いで決まって」おり、経済的な方針はほぼほぼ政党内で一致している
- 80議会の民主党は、やや政党内分裂が顕著だった
